In [2]:
! pip install forestci
! pip install duecredit
import forestci as fci
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 86 kB 2.5 MB/s 
     |████████████████████████████████| 179 kB 35.6 MB/s 


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf 
from tensorflow import keras

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# ML MODELS
def build_model(train_df, test_df):
  # Train test split
  X_train= train_df.drop(columns=['FVC','Patient'])
  y_train= train_df['FVC']
  X_test= test_df.drop(columns=['FVC','Patient'])
  y_test= test_df['FVC']
  # SVR Regressor Model
  svr_regressor = SVR(kernel = 'rbf')
  svr_regressor.fit(X_train, y_train)
  y_pred_svr= svr_regressor.predict(X_test)
#     svr_score = svr_regressor.score(X_test,y_test)
#     print("MSE:", mean_squared_error(y_test, y_pred_svr))
  # svr_proba= svr_regressor.predict_proba(X_test)
  # RF Regressor Model
  rf_regressor = RandomForestRegressor(max_depth=8, random_state=0)
  rf_regressor.fit(X_train, np.ravel(y_train))
  rf_regressor.score(X_test,np.ravel(y_test))
  y_pred_rf= rf_regressor.predict(X_test)
  # rf_proba= rf_regressor.predict_proba(X_test)
  mpg_V_IJ_unbiased = fci.random_forest_error(rf_regressor, X_train, X_test)
  # make FVC and Proba as Dataframes
  model_df= pd.DataFrame({'FVC_pred_svr':y_pred_svr,'FVC_pred_rf':y_pred_rf, 'Conf_svr':np.nan *len(y_pred_svr),'Conf_rf':np.nan *len(y_pred_svr)})
  return mpg_V_IJ_unbiased, model_df

In [21]:
def evluate(df_test, model_df):

    #Loglaplace
    FVC_true= df_test['FVC'].tolist()
    FVC_predicted_svr = model_df['FVC_pred_svr'].tolist()
    FVC_predicted_rf = model_df['FVC_pred_rf'].tolist()
    confidence_svr = model_df['Conf_svr'].tolist()
    confidence_rf = model_df['Conf_rf'].tolist()

    my_metric_list_svr= []
    for (i, j, k) in zip(FVC_true, FVC_predicted_svr, confidence_svr):
        # sd_clipped= max(k, 70)
        sd_clipped= 70
        delta= min(abs(i - j), 1000)
        metric= -((np.sqrt(2)*delta)/sd_clipped)  - np.log(np.sqrt(2)*sd_clipped)
        my_metric_list_svr.append(metric)
    log_laplace_metric_svr= np.mean(my_metric_list_svr)
    
    my_metric_list_rf= []
    for (i, j, k) in zip(FVC_true, FVC_predicted_rf, confidence_rf):
        # sd_clipped= max(k, 70)
        sd_clipped= 70
        delta= min(abs(i - j), 1000)
        metric= -((np.sqrt(2)*delta)/sd_clipped)  - np.log(np.sqrt(2)*sd_clipped)
        my_metric_list_rf.append(metric)
    log_laplace_metric_rf= np.mean(my_metric_list_rf)
    
    #MSE
    MSE_svr= mean_squared_error(FVC_true, FVC_predicted_svr)
    MSE_rf= mean_squared_error(FVC_true, FVC_predicted_rf)
    
    #MAPE : best value '0'
    #doubt: if need top pass 'multioutput' parameter in the function
    MAPE_svr= mean_absolute_percentage_error(FVC_true, FVC_predicted_svr)
    MAPE_rf= mean_absolute_percentage_error(FVC_true, FVC_predicted_rf)
    
    #MAE: best value '0'
    #doubt: if need top pass 'multioutput' parameter in the function
    MAE_svr= mean_absolute_error(FVC_true, FVC_predicted_svr)
    MAE_rf= mean_absolute_error(FVC_true, FVC_predicted_rf)
    print(MAE_rf)
    return (log_laplace_metric_svr,log_laplace_metric_rf)

In [22]:
Main_train=pd.read_csv("/content/drive/MyDrive/Main_train.csv")
Main_test=pd.read_csv("/content/drive/MyDrive/Main_test.csv")
Main_train.dropna(inplace=True)
Main_test.dropna(inplace=True)

In [23]:
print(Main_train)

                        Patient  Weeks   FVC    Percent  Age  \
0     ID00007637202177411956430     -4  2315  58.253649   79   
1     ID00007637202177411956430      5  2214  55.712129   79   
2     ID00007637202177411956430      7  2061  51.862104   79   
3     ID00007637202177411956430      9  2144  53.950679   79   
4     ID00007637202177411956430     11  2069  52.063412   79   
...                         ...    ...   ...        ...  ...   
1312  ID00426637202313170790466     19  2978  73.126412   73   
1313  ID00426637202313170790466     31  2908  71.407524   73   
1314  ID00426637202313170790466     43  2975  73.052745   73   
1315  ID00426637202313170790466     59  2774  68.117081   73   
1316  ID00426637202313170790466      0  2925  71.824968   73   

      SmokingStatus_Ex-smoker  SmokingStatus_Never smoked  Sex_Male  \
0                           1                           0         1   
1                           1                           0         1   
2                 

In [24]:
_,m=build_model(Main_train, Main_test)
print(evluate(Main_test, m))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature n

398.00655102270406
(-17.854055976917603, -12.546079147095792)
